In [ ]:
# 必要なライブラリのインストール（Google Colab用）
!pip install gspread oauth2client

import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

# Googleスプレッドシート認証
def connect_to_google_sheet():
    credentials_path = "/content/drive/MyDrive/credentials.json"  # 必要に応じて修正
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    try:
        credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)
        gc = gspread.authorize(credentials)
        spreadsheet = gc.open("タスク管理ツール")  # 任意のスプレッドシート名に変更
        return spreadsheet.sheet1  # 最初のシートを取得
    except FileNotFoundError:
        print("エラー: credentials.json ファイルが見つかりません。Google Driveに正しくアップロードされているか確認してください。")
        raise
    except Exception as e:
        print(f"予期しないエラーが発生しました: {e}")
        raise

# スプレッドシートのタスクを削除
def delete_task_from_sheet(sheet, row_number):
    try:
        if row_number > 1:  # ヘッダーは削除しないようにする
            sheet.delete_rows(row_number)
            print(f"スプレッドシートの{row_number}行目のタスクを削除しました。")
        else:
            print("エラー: 無効な行番号です。")
    except Exception as e:
        print(f"スプレッドシートのタスク削除中にエラーが発生しました: {e}")

# スプレッドシートからタスクを取得
def load_tasks_from_sheet(sheet):
    tasks = []
    rows = sheet.get_all_values()[1:]  # ヘッダーを除外

    for i, row in enumerate(rows, start=2):  # 2行目以降がデータ
        title = row[0]
        description = row[1]
        due_date = datetime.datetime.strptime(row[2], "%Y/%m/%d").date() if row[2] != "未設定" else None
        priority = row[3]
        completed = row[4] == "TRUE"
        task = Task(title, description, due_date, priority)
        task.completed = completed
        task.sheet_row = i  # スプレッドシート上の行番号
        tasks.append(task)

    return tasks

# タスククラス
class Task:
    def __init__(self, title, description="", due_date=None, priority="普通"):
        self.title = title
        self.description = description
        self.due_date = due_date
        self.priority = priority
        self.completed = False
        self.sheet_row = None

    def __str__(self):
        status = "完了" if self.completed else "未完了"
        due_date_str = self.due_date.strftime("%Y/%m/%d") if self.due_date else "未設定"
        return f"タスク: {self.title} ({self.priority})\n説明: {self.description}\n期限: {due_date_str}\n状態: {status}\n"

    def mark_completed(self):
        self.completed = True

def add_task(tasks):
    title = input("タスク名を入力してください: ")
    description = input("説明を入力してください (省略可): ")
    due_date_str = input("期限を入力してください (YYYY/MM/DD、省略可): ")
    priority = input("優先度を入力してください (高, 中、低、普通、省略可): ")

    due_date = None
    if due_date_str:
        try:
            due_date = datetime.datetime.strptime(due_date_str, "%Y/%m/%d").date()
        except ValueError:
            print("日付の形式が正しくありません。YYYY/MM/DDで入力してください。")
            return

    if not priority:
        priority = "普通"
    elif priority not in ["高", "中", "低", "普通"]:
        print("優先度は高、中、低、普通のいずれかを入力してください")
        return

    tasks.append(Task(title, description, due_date, priority))
    print("タスクを追加しました。")

def list_tasks(tasks, sheet):
    sheet_tasks = load_tasks_from_sheet(sheet)
    all_tasks = tasks + sheet_tasks

    if not all_tasks:
        print("タスクはありません。")
        return

    print("=== タスク一覧 ===")
    for i, task in enumerate(all_tasks):
        print(f"{i+1}. {task}")

def complete_task(tasks, sheet):
    sheet_tasks = load_tasks_from_sheet(sheet)
    all_tasks = tasks + sheet_tasks

    list_tasks(tasks, sheet)

    try:
        task_index = int(input("完了したタスクの番号を入力してください: ")) - 1
        if 0 <= task_index < len(all_tasks):
            selected_task = all_tasks[task_index]
            selected_task.mark_completed()
            print("タスクを完了済みにしました。")

            delete_choice = input("このタスクを削除しますか？ (y/n): ").lower()
            if delete_choice == "y":
                if selected_task in tasks:
                    tasks.remove(selected_task)
                    print("ローカルタスクを削除しました。")
                elif selected_task in sheet_tasks:
                    delete_task_from_sheet(sheet, selected_task.sheet_row)
        else:
            print("無効なタスク番号です。")
    except ValueError:
        print("無効な入力です。")

def save_tasks_to_sheet(tasks, sheet):
    existing_data = sheet.get_all_values()[1:]
    existing_titles = [row[0] for row in existing_data]

    for task in tasks:
        if task.title not in existing_titles:
            due_date_str = task.due_date.strftime("%Y/%m/%d") if task.due_date else "未設定"
            status = "TRUE" if task.completed else "FALSE"
            sheet.append_row([task.title, task.description, due_date_str, task.priority, status])

    print("タスクをGoogleスプレッドシートに保存しました。")

def main():
    tasks = []
    sheet = connect_to_google_sheet()

    while True:
        print("\nタスク管理ツール")
        print("1. タスクを追加")
        print("2. タスク一覧を表示")
        print("3. タスクを完了")
        print("4. Googleスプレッドシートに保存")
        print("5. 終了")

        choice = input("選択肢を入力してください: ")

        if choice == "1":
            add_task(tasks)
        elif choice == "2":
            list_tasks(tasks, sheet)
        elif choice == "3":
            complete_task(tasks, sheet)
        elif choice == "4":
            save_tasks_to_sheet(tasks, sheet)
        elif choice == "5":
            break
        else:
            print("無効な選択肢です。")

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

タスク管理ツール
1. タスクを追加
2. タスク一覧を表示
3. タスクを完了
4. Googleスプレッドシートに保存
5. 終了
選択肢を入力してください: 2
=== タスク一覧 ===
1. タスク: テストテストテスト (普通)
説明: 
期限: 未設定
状態: 完了

2. タスク: テスト追加 (普通)
説明: 
期限: 未設定
状態: 未完了

3. タスク: test (高)
説明: 
期限: 未設定
状態: 未完了

4. タスク: testtest (普通)
説明: 
期限: 未設定
状態: 完了

5. タスク: testtest (普通)
説明: 
期限: 未設定
状態: 未完了

6. タスク: testtest (普通)
説明: 
期限: 未設定
状態: 未完了

7. タスク: testtest (普通)
説明: 
期限: 未設定
状態: 完了


タスク管理ツール
1. タスクを追加
2. タスク一覧を表示
3. タスクを完了
4. Googleスプレッドシートに保存
5. 終了
選択肢を入力してください: 3
=== タスク一覧 ===
1. タスク: テストテストテスト (普通)
説明: 
期限: 未設定
状態: 完了

2. タスク: テスト追加 (普通)
説明: 
期限: 未設定
状態: 未完了

3. タスク: test (高)
説明: 
期限: 未設定
状態: 未完了

4. タスク: testtest (普通)
説明: 
期限: 未設定
状態: 完了

5. タスク: testtest (普通)
説明: 
期限: 未設定
状態: 未完了

6. タスク: testtest (普通)
説明: 
期限: 未設定
状態: 未完了

7. タスク: testtest (普通)
説明: 
期限: 未設定
状態: 完了

完了したタスクの番号を入力してください: 2
タスクを完了済みにしました。
このタスクを削除しますか？ (y/n): y
スプレッドシートの3行目のタスク